In [1]:
# Create a DAG to run the ETL process for the leads data consuming the data from the leads REST API
## The API connection ID is pipe_leads returns a JSON on an object with the following structure: 'data'

#from airflow.providers.mysql.operators.mysql import MySqlOperator
#from airflow.operators.dummy_operator import DummyOperator
#from airflow.providers.mysql.hooks.mysql import MySqlHook
from datetime import datetime, timedelta
#from airflow.decorators import dag, task
#from airflow.models import Variable
from mysql.connector import Error
from dateutil import parser
import mysql.connector
import calendar
import requests
import base64
import json
import time
import re

In [2]:
# Retrieve database configuration from Airflow Variable
#aircalls_cron_schedule = Variable.get("aircalls_cron_schedule", default_var="*/2 * * * *")
api_test = 'True'
api_limit = 500
api_start = 0
##api_token = Variable.get("pipedrive_api_token")
api_aircalls_url = 'https://api.aircall.io/v1/calls?order=desc&per_page=50'
api_user = '36f1f8c50b6d4e5fdcdd5f06db496b69'
api_pass = '0627dc5681942153e6ac0788ebd3340a'
batch_size = 100
mysql_conn_id = 'my_sql'

In [3]:
connection = mysql.connector.connect(
            user = 'pipedrive',
            password = '#8LsH25%ZD',
            host = 'scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
            database= 'pipedrive')

In [4]:
def format_raw_digits(raw_digits):
    if raw_digits is None:
        return None
    # Remove all spaces from the string
    return raw_digits.replace(" ", "")

In [5]:
def insert_update_number(number_data):
    connection = mysql.connector.connect(
            user = 'pipedrive',
            password = '#8LsH25%ZD',
            host = 'scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
            database= 'pipedrive')
    #print("Inserting/Updating number:", number_data)
    # Extract data from the number_data dictionary
    number_id = number_data.get("id")
    direct_link = number_data.get("direct_link")
    name = number_data.get("name")
    digits = number_data.get("digits")
    country = number_data.get("country")
    time_zone = number_data.get("time_zone")
    open = number_data.get("open")
    availability_status = number_data.get("availability_status")
    is_ivr = number_data.get("is_ivr")
    live_recording_activated = number_data.get("live_recording_activated")
    priority = number_data.get("priority")
    created_at = number_data.get("created_at")
    created_at_datetime = None
    if created_at is not None:
        # Parse the ISO formatted datetime string to a datetime object
        created_at_datetime = parser.parse(created_at)

    values = ( number_id, direct_link, name, digits, country, time_zone, open, availability_status, is_ivr, live_recording_activated, priority, created_at_datetime)
    
    script_insert_or_update_users = """
    INSERT INTO aircalls_users_dag (id, direct_link, name, email, available, availability_status, created_at, time_zone, language, wrap_up_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE 
                direct_link = VALUES(direct_link), 
                name = VALUES(name), 
                email = VALUES(email), 
                available = VALUES(available), 
                availability_status = VALUES(availability_status), 
                created_at = VALUES(created_at), 
                time_zone = VALUES(time_zone), 
                language = VALUES(language), 
                wrap_up_time = VALUES(wrap_up_time);
    """
    try:
        cursor = connection.cursor()
        cursor.execute(script_insert_or_update_users, values)
        connection.commit()
       # print("User insert/update successful.")
        #print(f"User Values: {values}")
    except mysql.connector.Error as error:
        connection.rollback()
    # We need to close the cursor and the connection, and this needs to be done regardless of what happened above.
    return number_id

In [23]:
def insert_update_user(user_data):
    connection = mysql.connector.connect(
            user = 'pipedrive',
            password = '#8LsH25%ZD',
            host = 'scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
            database= 'pipedrive')
    #print("Inserting/Updating user:", user_data)
    # Extract data from the user_data dictionary
    user_id = user_data.get("id")
    direct_link = user_data.get("direct_link")
    name = user_data.get("name")
    email = user_data.get("email")
    available = user_data.get("available")
    availability_status = user_data.get("availability_status")
    created_at = user_data.get("created_at")
    created_at_datetime = None
    if created_at is not None:
        # Parse the ISO formatted datetime string to a datetime object
        created_at_datetime = parser.parse(created_at)
    time_zone = user_data.get("time_zone")
    language = user_data.get("language")
    wrap_up_time = user_data.get("wrap_up_time")

    values = (user_id, direct_link, name, email, available, availability_status, created_at_datetime, time_zone, language, wrap_up_time)
    
    script_insert_or_update_users = """
    INSERT INTO aircalls_users_dag (id, direct_link, name, email, available, availability_status, created_at, time_zone, language, wrap_up_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE 
    direct_link = VALUES(direct_link), 
    name = VALUES(name), 
    email = VALUES(email), 
    available = VALUES(available), 
    availability_status = VALUES(availability_status), 
    created_at = VALUES(created_at), 
    time_zone = VALUES(time_zone), 
    language = VALUES(language), 
    wrap_up_time = VALUES(wrap_up_time);
    """
    try:
        cursor = connection.cursor()
        cursor.execute(script_insert_or_update_users, values)
        connection.commit()
        #print("User insert/update successful.")
        #print(f"User Values: {values}")
    except mysql.connector.Error as error:
        connection.rollback()
        print("Error: {}")
    # We need to close the cursor and the connection, and this needs to be done regardless of what happened above.
    return user_id

In [24]:
def insert_update_call(call_data):
    
    
    #print("Inserting/Updating call:", call_data)
    # Extract data from the call_data dictionary
    call_id = call_data.get("id")
    direct_link = call_data.get("direct_link")
    direction = call_data.get("direction")
    status = call_data.get("status")
    missed_call_reason = call_data.get("missed_call_reason")
    started_at = call_data.get("started_at")
    started_at_datetime = None
    if started_at is not None:
        # Parse the ISO formatted datetime string to a datetime object
        started_at_datetime = datetime.fromtimestamp(started_at)
    answered_at = call_data.get("answered_at")
    answered_at_datetime = None
    if answered_at is not None:
        # Parse the ISO formatted datetime string to a datetime object
        answered_at_datetime = datetime.fromtimestamp(answered_at)
    ended_at = call_data.get("ended_at")
    ended_at_datetime = None
    if ended_at is not None:
        # Parse the ISO formatted datetime string to a datetime object
        ended_at_datetime = datetime.fromtimestamp(ended_at)
    duration = call_data.get("duration")
    voicemail = call_data.get("voicemail")
    asset = call_data.get("asset")
    raw_digits = format_raw_digits(call_data.get("raw_digits"))
    # Validate and insert/update the user
    if call_data.get("user") is not None:
        user_id = None
        user_id = insert_update_user(call_data.get("user"))
    else:
        user_id = None
    archived = call_data.get("archived")
    assigned_to = call_data.get("assigned_to")
    transferred_by = call_data.get("transferred_by")
    transferred_to = call_data.get("transferred_to")
    # Validate and insert/update the number
    if call_data.get("number") is not None:
        number_id = None
        number_id = insert_update_number(call_data.get("number"))
    else:
        number_id = None

    cost = call_data.get("cost")
    country_code_a2 = call_data.get("country_code_a2")
    pricing_type = call_data.get("pricing_type")

    values = (call_id, direct_link, direction, status, missed_call_reason, started_at_datetime, answered_at_datetime,
        ended_at_datetime, duration, voicemail, asset, raw_digits, user_id, archived, assigned_to, transferred_by,
        transferred_to, number_id, cost, country_code_a2, pricing_type)
    
    script_insert_or_update_calls = """
    INSERT INTO aircalls_calls_dag (id, direct_link, direction, status, missed_call_reason, started_at, answered_at, ended_at, duration, voicemail, asset, raw_digits, user_id, archived, assigned_to, transferred_by, transferred_to, number_id, cost, country_code_a2, pricing_type) 
    VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s) ON DUPLICATE KEY UPDATE 
                direct_link = VALUES(direct_link),
                direction = VALUES(direction),
                status = VALUES(status),
                missed_call_reason = VALUES(missed_call_reason),
                started_at = VALUES(started_at),
                answered_at = VALUES(answered_at),
                ended_at = VALUES(ended_at),
                duration = VALUES(duration),
                voicemail = VALUES(voicemail),
                asset = VALUES(asset),
                raw_digits = VALUES(raw_digits),
                user_id = VALUES(user_id),
                archived = VALUES(archived),
                assigned_to = VALUES(assigned_to),
                transferred_by = VALUES(transferred_by),
                transferred_to = VALUES(transferred_to),
                number_id = VALUES(number_id),
                cost = VALUES(cost),
                country_code_a2 = VALUES(country_code_a2),
                pricing_type = VALUES(pricing_type);
                """                              
    try:
        cursor = connection.cursor()
        cursor.execute(script_insert_or_update_calls, values)
        connection.commit()
        print(f"Call insert/update successful: {started_at_datetime}")
        #print(f"Call Values: {values}")
        
    except mysql.connector.Error as error:
        connection.rollback()
        print("Error: {}. not updated!".format(error))
    # We need to close the cursor and the connection, and this needs to be done regardless of what happened above.

In [25]:
def get_primary_contact(contacts):
    print("Extracting primary contact from:", contacts)
    for contact in contacts:
        if contact.get("primary", False):
            primary_contact = contact.get("value")
            #print(f"Primary contact found: {primary_contact}")
            return primary_contact
    print("No primary contact found.")
    return None

In [26]:
def format_phone_number(phone_number):
    print(f"Formatting phone number: {phone_number}")
    if not phone_number:
        print("No phone number provided.")
        return None  # Or an empty string or any default value you prefer

    cleaned_number = re.sub(r"(?!^\+)\D", "", phone_number)
    #print(f"Cleaned number: {cleaned_number}")

    if cleaned_number.startswith("+"):
        return cleaned_number
    elif cleaned_number.startswith("800"):
        return cleaned_number
    elif cleaned_number.startswith("39"):
        return "+" + cleaned_number
    elif not cleaned_number.startswith("39") and len(cleaned_number) <= 10:
        return "+39" + cleaned_number
    else:
        return "+" + cleaned_number

In [27]:
def parse_date_time(data, field_name):
    raw_date_time = data.get(field_name)
    print(f"Parsing date/time for {field_name}: {raw_date_time}")
    if raw_date_time:
        try:
            parsed_date_time = datetime.strptime(raw_date_time, "%Y-%m-%d %H:%M:%S")
            #print(f"Parsed date/time: {parsed_date_time}")
            return parsed_date_time
        except ValueError as e:
            print(f"Error parsing date/time for {field_name}: {e}")
    return None

In [28]:
def parse_date(data, field_name):
    raw_date = data.get(field_name)
    print(f"Attempting to parse date for '{field_name}': {raw_date}")
    if raw_date:
        try:
            parsed_date = datetime.strptime(raw_date, "%Y-%m-%d").date()
            #print(f"Parsed date for '{field_name}': {parsed_date}")
            return parsed_date
        except ValueError as e:
            print(f"Error parsing date for '{field_name}': {raw_date}, Error: {e}")
    else:
        print(f"No date found for '{field_name}'.")
    return None

In [29]:
def parse_double(data, field_name):
    value = data.get(field_name)
    print(f"Attempting to parse double for '{field_name}': {value}")
    try:
        if value is None:
            print(f"No value provided for '{field_name}', returning None.")
            return None
        parsed_double = float(value)
        #print(f"Parsed double for '{field_name}': {parsed_double}")
        return parsed_double
    except (TypeError, ValueError) as e:
        print(f"Error parsing double for '{field_name}': {value}, Error: {e}")
        return None

In [30]:
def datetime_to_unix_timestamp(date):
    return int(time.mktime(date.timetuple()))

In [31]:
def make_api_call_v2(url, user, passw):
    max_retries = 3
    retry_delay = 2  # seconds
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{user}:{passw}".encode()).decode()
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"API call failed, attempt {attempt + 1}. Retrying in {retry_delay} seconds.")
            if attempt + 1 == max_retries:
                raise
            time.sleep(retry_delay)

In [32]:
def process_response_and_insert_calls(response):
    # Assuming `response` is a JSON object similar to what's returned from an API call
    calls = response.get('calls', [])
    total_processed = 0

    # Initialize MySqlHook
    
#mysql_hook = MySqlHook(mysql_conn_id=mysql_conn_id)

    for call in calls:
        # Here, you would have the logic to insert or update the call data
        # For simplicity, we assume a function `insert_or_update_call_data` exists
        insert_update_call(call)
        total_processed += 1

    #print(f"Processed {total_processed} records")

    # Fetching the next page link from the response for pagination
    meta = response.get('meta', {})
    next_page_link = meta.get('next_page_link', None)
    print(f"Page processed, next page link: {next_page_link}")
    print(f"Current page: {meta.get('current_page')}")

    return next_page_link, bool(next_page_link)


In [33]:
# First DAG - Original
def fetch_all_calls():
    connection = mysql.connector.connect(
            user = 'pipedrive',
            password = '#8LsH25%ZD',
            host = 'scraper.cx53soegx3qk.eu-west-1.rds.amazonaws.com',
            database= 'pipedrive')
    start_date_str = "2024-04-02"
    end_date_str = "2024-05-04"
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    print(f"Start date: {start_date}, End date: {end_date}")
    current_date = start_date
    while current_date <= end_date:
        print("Processing date:", current_date)
        start_of_day = datetime(current_date.year, current_date.month, current_date.day)
        end_of_day = start_of_day + timedelta(days=1, seconds=-1)
        end_timestamp = datetime_to_unix_timestamp(end_of_day)
        start_timestamp = datetime_to_unix_timestamp(start_of_day)

        # Constructing the API call with date filters for the current day
        api_url = f"{api_aircalls_url}&from={start_timestamp}&to={end_timestamp}"
        print("API URL:", api_url)
        has_more_data = True
        try:
            while has_more_data:
                    response = make_api_call_v2(api_url, api_user, api_pass)
                    calls = response.get("calls", [])
                    meta = response.get("meta", {})
                    print(f"API meta repsonse: {meta}")
                    print(f'API calls to process: {len(calls)}')
                    # Process the response, insert data into the database, etc.
                    next_page_link, has_more_data = process_response_and_insert_calls(response)
            if next_page_link:
                    api_url = next_page_link
            else:
                    has_more_data = False
    
        except Exception as e:
            print(f"Failed to make API call: {e}")
        finally:
            if connection.is_connected():
                connection.close()
                print("MySQL connection is closed.")
        current_date += timedelta(days=1)

In [34]:
fetch_all_calls()